In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
path = '/content/drive/MyDrive/Colab Notebooks/NeurIPS2024'
sys.path.append(path)

import numpy as np
import pandas as pd
import pickle

from realdata import *

import os

import matplotlib.pyplot as plt


Mounted at /content/drive


In [4]:
algo = 'XGB'

test_next = False


results_all = []
for idx in range(10):
    le = idx * 10 + 1
    ri = le + 9
    filename = path + '/results/{}_{}_{}_{}.pkl'.format(algo, le, ri, test_next)
    with open(filename, 'rb') as r:
        tmp = pickle.load(r)
        results_all += tmp


D = 10
M = int(np.ceil(826 / D))
list_t = np.array(range(1, M)) * D

prop = [0.3, 0.1]
window_sizes_train = [1, 16, 256, 1024]
window_sizes_infer = [1, 4, 16, 64, 256, 1024]
weights = [0.99, 0.9, 0.5, 0.25]
alpha = 0.1
delta = 0.1



num_train = len(window_sizes_train)
num_infer = len(window_sizes_infer)
num_weights = len(weights)
num_t = len(list_t)


S = len(results_all)
idx_start = 0
print('Algorithm = {}'.format(algo))
print()
print()

for (idx_model, window_size) in enumerate(window_sizes_train):
    print('Results for training window {}: '.format(window_sizes_train[idx_model]))

    MAE = dict()
    MAE['ARW'] = 0
    MAE['weighted'] = np.zeros(num_weights)
    MAE['fixed'] = np.zeros(num_infer)

    MAEsd = dict()
    MAEsd['ARW'] = 0
    MAEsd['weighted'] = np.zeros(num_weights)
    MAEsd['fixed'] = np.zeros(num_infer)

    AC = dict()
    AC['ARW'] = 0
    AC['weighted'] = np.zeros(num_weights)
    AC['fixed'] = np.zeros(num_infer)

    AW = dict()
    AW['ARW'] = 0
    AW['weighted'] = np.zeros(num_weights)
    AW['fixed'] = np.zeros(num_infer)
    AW['baseline'] = 0


    for s in range(S):
        MAE_s, AC_s, AW_s = summarize(results_all[s], alpha, idx_start)
        for key in MAE.keys():
            MAE[key] += MAE_s[idx_model][key] / S
            MAEsd[key] += MAE_s[idx_model][key] ** 2 / S
            AC[key] += AC_s[idx_model][key] / S
        for key in AW.keys():
            AW[key] += AW_s[idx_model][key] / S

    for key in MAE.keys():
        MAEsd[key] = np.sqrt(MAEsd[key] - (MAE[key] ** 2))

    print('MAE: ', MAE)
    print('AC: ', AC)
    print('AW: ', AW)

    print()

    print('MAEsd: ', MAEsd)

    print()
    print()


Algorithm = XGB


Results for training window 1: 
MAE:  {'ARW': 0.03439104349878602, 'weighted': array([0.04471183, 0.04472783, 0.04522525, 0.04587353]), 'fixed': array([0.06276296, 0.03715504, 0.03020688, 0.03209009, 0.04141216,
       0.04471183])}
AC:  {'ARW': 0.9252498405799425, 'weighted': array([0.93947505, 0.93950212, 0.94023891, 0.94110887]), 'fixed': array([0.87447498, 0.90322343, 0.91043186, 0.91875444, 0.93424118,
       0.93947505])}
AW:  {'ARW': 1.5815419967472326, 'weighted': array([1.69545474, 1.6957338 , 1.70171066, 1.70967351]), 'fixed': array([1.376253  , 1.46953747, 1.49396683, 1.54772469, 1.65233261,
       1.69545474]), 'baseline': 2.175496702154525}

MAEsd:  {'ARW': 0.0021290006683297802, 'weighted': array([0.00213687, 0.00214094, 0.00215127, 0.00220674]), 'fixed': array([0.00576099, 0.00337593, 0.00239798, 0.0022651 , 0.00238756,
       0.00213687])}


Results for training window 16: 
MAE:  {'ARW': 0.03136814056285026, 'weighted': array([0.06177693, 0.06180184, 0